<a href="https://colab.research.google.com/github/maddivikash/EPL-predictions/blob/main/Predicting_Match_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv("/content/epl_matches_train.csv")

In [3]:
data['result'] = data['home_team_goal']  - data['away_team_goal'] 

In [4]:
data['r'] = 'win'
for r in data.index:
    if(int(data['result'][r]) >0 ):
        data['r'][r] = 'win'
        continue
    if(int(data['result'][r]) ==0 ):
        data['r'][r] = 'draw'
    else:
        data['r'][r] =  'lose'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [5]:

data = data.fillna(method = "ffill")

In [24]:
data = data.fillna(method = "ffill")
X_all = data.drop(['result','r','season','date','home_team_goal','away_team_goal'],axis = 1)
X_all = X_all.drop(X_all.iloc[:, -16:], axis = 1)
y_all = data['r']


In [25]:
def preprocess_features(X):
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

In [26]:
from sklearn.preprocessing import scale
cols = X_all.columns
for col in cols:
    X_all[col] = scale(X_all[col])

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 30,
                                                    random_state = 2,
                                                    stratify = y_all)

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [29]:
clf1 = RandomForestClassifier()
clf2 = MultinomialNB()
clf3 = XGBClassifier()
clf4 = LinearSVC()

In [30]:

model1 = LogisticRegression(max_iter=10000)
model1.fit(X_train, y_train)
predicted = model1.predict(X_test)
predicted
acc_score = accuracy_score(y_test, predicted)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(model1, X_train, y_train, cv=10)
print("Cross val scores")
print(scores)

Accuracy score: 0.43333333333333335
Cross val scores
[0.47908745 0.48669202 0.44106464 0.46768061 0.42205323 0.47528517
 0.46768061 0.4904943  0.45247148 0.44106464]


In [31]:
model = XGBClassifier(max_iter=10000)
model.fit(X_all, y_all)
model.score(X_all, y_all)

0.6680451127819549

In [32]:

model1 = XGBClassifier(max_iter=10000)
model1.fit(X_train, y_train)
predicted = model1.predict(X_test)
predicted
acc_score = accuracy_score(y_test, predicted)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(model1, X_train, y_train, cv=10)
print("Cross val scores")
print(scores)

Accuracy score: 0.5
Cross val scores
[0.5095057  0.4904943  0.47908745 0.5095057  0.48669202 0.48288973
 0.52471483 0.52091255 0.48288973 0.47908745]


In [16]:
data1 = pd.read_csv("/content/epl_matches_test.csv")

In [17]:
data1 = data1.fillna(method = "ffill")

In [18]:
data1 = data1.fillna(method = "ffill")
X1_all = data1.drop(['season','date'],axis = 1)

In [19]:
def preprocess_features(X):
    output = pd.DataFrame(index = X.index)
    for col, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
        output = output.join(col_data)
    
    return output

X1_all = preprocess_features(X1_all)

In [20]:
from sklearn.preprocessing import scale
cols = X1_all.columns
for col in cols:
  if(col != 'match_id'):
    X1_all[col] = scale(X1_all[col])

In [21]:
data1['match_id']

0      48994
1      54777
2      31434
3      56248
4      44311
       ...  
375    42896
376    49098
377    55212
378    54075
379    40908
Name: match_id, Length: 380, dtype: int64

In [23]:
prediction1 = pd.DataFrame()
prediction1['match_id'] = data1['match_id']

prediction1['result'] = model.predict(X1_all)
prediction1.to_csv('prediction1.csv', index=False)
prediction1

,match_id,result
0,48994,lose
1,54777,win
2,31434,lose
3,56248,lose
4,44311,lose
...,...,...
375,42896,lose
376,49098,lose
377,55212,lose
378,54075,lose
